In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

In [ ]:
CSV_PATH = "path/to/train.csv"
IMG_ROOT = "path/to/image/folder"  # os.path.join(IMG_ROOT, image_path)
BATCH_SIZE = 32
NUM_WORKERS = 2
NUM_EPOCHS = 10
LR = 1e-4
IMG_SIZE = 224

In [ ]:
df = pd.read_csv(CSV_PATH)

label_encoder = LabelEncoder()
df["class_encoded"] = label_encoder.fit_transform(df["class"])
num_classes = len(label_encoder.classes_)
print(f"Total Classes: {num_classes}")

In [ ]:
class FoodDataset(Dataset):
    def __init__(self, df, img_root, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_root = img_root
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_root, row["Image"])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label_class = row["class_encoded"]
        label_mos = row["mos"]

        return image, torch.tensor(label_class), torch.tensor(label_mos, dtype=torch.float32)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["class_encoded"], random_state=42)

train_dataset = FoodDataset(train_df, IMG_ROOT, transform)
val_dataset = FoodDataset(val_df, IMG_ROOT, transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [ ]:
class MultiTaskNet(nn.Module):
    def __init__(self, num_classes):
        super(MultiTaskNet, self).__init__()
        base = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(base.children())[:-1])
        self.flatten = nn.Flatten()
        self.class_head = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        self.reg_head = nn.Sequential(
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.flatten(x)
        return self.class_head(x), self.reg_head(x).squeeze(1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiTaskNet(num_classes).to(device)

criterion_class = nn.CrossEntropyLoss()
criterion_reg = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for x, y_class, y_reg in tqdm(train_loader):
        x, y_class, y_reg = x.to(device), y_class.to(device), y_reg.to(device)

        pred_class, pred_reg = model(x)

        loss_class = criterion_class(pred_class, y_class)
        loss_reg = criterion_reg(pred_reg, y_reg)
        loss = loss_class + loss_reg

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")

In [ ]:
model.eval()
correct = 0
total = 0
mae = 0

with torch.no_grad():
    for x, y_class, y_reg in val_loader:
        x, y_class, y_reg = x.to(device), y_class.to(device), y_reg.to(device)

        pred_class, pred_reg = model(x)
        predicted = pred_class.argmax(1)
        correct += (predicted == y_class).sum().item()
        total += y_class.size(0)

        mae += torch.abs(pred_reg - y_reg).sum().item()

accuracy = correct / total
mae = mae / total

print(f"Validation Accuracy: {accuracy:.4f}, MAE (mos): {mae:.4f}")

In [ ]:
submission_df = pd.read_csv("path/to/submission.csv")
submission_dataset = FoodDataset(submission_df, IMG_ROOT, transform)
submission_loader = DataLoader(submission_dataset, batch_size=BATCH_SIZE, shuffle=False)

model.eval()
predicted_classes = []
predicted_mos = []

with torch.no_grad():
    for x, _, _ in submission_loader:
        x = x.to(device)
        pred_class, pred_reg = model(x)

        predicted_classes.extend(pred_class.argmax(1).cpu().numpy())
        predicted_mos.extend(pred_reg.cpu().numpy())

submission_df["MOS"] = predicted_mos
submission_df["Class"] = label_encoder.inverse_transform(predicted_classes)

submission_df.to_csv("submission_result.csv", index=False)
print("Submission saved to submission_result.csv")